In [19]:
# @title Environment Setup
import os
import sys

# 1. *** FIX: Clear problematic environment variable for matplotlib ***
# This prevents the "ValueError: Key backend: 'module://matplotlib_inline.backend_inline'" error
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
!rm -rf FontDiffusion
!git clone https://github.com/dzungphieuluuky/FontDiffusion.git

!uv pip install --upgrade pip
!uv pip install -r FontDiffusion/requirements.txt
!uv pip install gdown
# 3. Install PyTorch 1.13
print("\n⬇️ Installing PyTorch 1.13 (Required for this model)...")
# Force reinstall torch 1.13 to match the model's training environment
!uv pip uninstall torch torchvision
!uv pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117

# 4. Install other dependencies
print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!uv pip install xformers==0.0.16 -q

# Install original dependencies
!uv pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!uv pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
# -----------------------------------------------------------------
!uv pip install huggingface-hub>=0.15.1,<1.0
!uv pip install lpips scikit-image pytorch-fid
!sudo apt-get update && sudo apt-get install dos2unix
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

Cloning into 'FontDiffusion'...
remote: Enumerating objects: 15094, done.
remote: Counting objects: 100% (2907/2907), done.
remote: Compressing objects: 100% (2825/2825), done.
remote: Total 15094 (delta 111), reused 2863 (delta 80), pack-reused 12187 (from 3)
Receiving objects: 100% (15094/15094), 246.94 MiB | 23.48 MiB/s, done.
Resolving deltas: 100% (499/499), done.
Updating files: 100% (122/122), done.
Using Python 3.11.13 environment at: /usr
Resolved 1 package in 45ms                                           
Audited 1 package in 0.18ms
Using Python 3.11.13 environment at: /usr
Audited 24 packages in 116ms
Using Python 3.11.13 environment at: /usr
Audited 1 package in 101ms

⬇️ Installing PyTorch 1.13 (Required for this model)...
Using Python 3.11.13 environment at: /usr
Uninstalled 2 packages in 341ms
 - torch==2.9.1
 - torchvision==0.24.1
Using Python 3.11.13 environment at: /usr
  × No solution found when resolving dependencies:                                  
  ╰─▶ Because

In [21]:
# 1. Create a new Conda environment with Python 3.9
# This handles the Linux + Python 3.9 requirement
!conda create -n font_env python=3.9 -y

# 2. Install PyTorch 1.13.1 with CUDA 11.7 binaries
# We use the specific pytorch channel to ensure compatibility
!/opt/conda/envs/font_env/bin/pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117

# 3. Install the FontDiffusion dependencies and the specific huggingface version
# This prevents the 'cached_download' error you saw earlier
!/opt/conda/envs/font_env/bin/pip install huggingface_hub==0.19.4 diffusers==0.14.0 transformers accelerate

# 4. Verify the installation
print("--- Verification ---")
!/opt/conda/envs/font_env/bin/python -c "import torch; print(f'Python Version: {torch.__ext_schema__}'); print(f'PyTorch: {torch.__version__}'); print(f'CUDA available: {torch.cuda.is_available()}'); print(f'CUDA version: {torch.version.cuda}')"

/bin/bash: line 1: conda: command not found
/bin/bash: line 1: /opt/conda/envs/font_env/bin/pip: No such file or directory
/bin/bash: line 1: /opt/conda/envs/font_env/bin/pip: No such file or directory
--- Verification ---
/bin/bash: line 1: /opt/conda/envs/font_env/bin/python: No such file or directory


In [2]:
import os
from IPython import get_ipython

def configure_environment_paths():
    """Detect environment and configure paths"""
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"

    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")

    return base_data_path, base_output_path, environment_name


INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()

✅ Environment: Kaggle
📂 Data Path: /kaggle/input/
📦 Output Path: /kaggle/working/


In [3]:
import os
import wandb

if "colab" in ENV_NAME:
    from google.colab import userdata

    try:
        # Ensure 'WANDB_API_KEY' is the exact name in your Colab Secrets (the key icon)
        wandb_key = userdata.get("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Colab Secrets: {e}")

# 2. Check if running in Kaggle
elif "kaggle" in ENV_NAME:
    try:
        from kaggle_secrets import UserSecretsClient

        user_secrets = UserSecretsClient()
        wandb_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Kaggle Secrets: {e}")

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Could not retrieve W&B API key from Kaggle Secrets: Unexpected response from the service. Response: {'errors': ['No user secrets exist for kernel id 103951363 and label WANDB_API_KEY.'], 'error': {'code': 5}, 'wasSuccessful': False}.


In [4]:
import gdown

if not os.path.exists("ckpt"):
  url = "https://drive.google.com/drive/folders/12hfuZ9MQvXqcteNuz7JQ2B_mUcTr-5jZ"
  gdown.download_folder(url, quiet=True, use_cookies=False)

In [5]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_PATH, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_PATH}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -q -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

No .zip files found in /kaggle/input/.


In [6]:
# @title Checking checkpoint files (.pth)
import os
import time

CHECKPOINT_DIR = os.path.join(INPUT_PATH, "ckpt")
print(CHECKPOINT_DIR)
# Create the checkpoint directory
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
# Wait loop to check if files exist
required_files = ["unet.pth", "content_encoder.pth", "style_encoder.pth"]

while True:
    missing = [f for f in required_files if not os.path.exists(f"{CHECKPOINT_DIR}/{f}")]

    if not missing:
        print("\n✅ All weights found! You can proceed to the next step.")
        break
    else:
        print(f"Waiting for files... Missing: {missing}")
        print("Upload them to the 'ckpt' folder now.")
        time.sleep(10) # Checks every 10 seconds

/kaggle/input/ckpt


OSError: [Errno 30] Read-only file system: '/kaggle/input/ckpt'

In [7]:
import pandas as pd
nom_tu_tao_300_df = pd.read_csv(f"{INPUT_PATH}/Ds_300_ChuNom_TuTao.csv")
nom_tu_tao = nom_tu_tao_300_df['word'].tolist()

with open(f"{OUTPUT_PATH}/nom_tu_tao.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(nom_tu_tao))

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input//Ds_300_ChuNom_TuTao.csv'

In [20]:
%cd {OUTPUT_PATH}/FontDiffusion
!python batch_sample_evaluate.py \
    --characters "NomTuTao/Ds_10k_ChuNom_TuTao.txt" \
    --start_line 1 \
    --end_line 100 \
    --style_images "style_images/" \
    --ckpt_dir "../ckpt/" \
    --ttf_path "fonts/" \
    --output_dir "../output_1-100_10k" \
    --batch_size 8 \
    --save_interval 5 \
    --fp16 \
    --channels_last

/kaggle/working/FontDiffusion
/bin/bash: line 1: /opt/conda/envs/colab_style_env/bin/python: No such file or directory


In [ ]:
import os
import glob
import zipfile
from typing import List
def find_result_folders(base_path: str) -> List[str]:
    """
    Return a list of absolute paths to all directories under `base_path`
    whose names start with 'results_'.
    """
    pattern = os.path.join(base_path, "output_*")
    # glob returns both files and directories; filter to directories only
    return [p for p in glob.glob(pattern) if os.path.isdir(p)]
def zip_folder(folder_path: str, output_base_path: str) -> bool:
    """
    Zip the contents of `folder_path` into a file named
    <folder_name>.zip` inside `output_base_path`.

    Returns True on success, False otherwise.
    """
    folder_name = os.path.basename(folder_path)
    zip_path = os.path.join(output_base_path, f"{folder_name}.zip")
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(
            zip_path, mode="w", compression=zipfile.ZIP_DEFLATED
        ) as zipf:
            for root, _, files in os.walk(folder_path):
                for file in files:
                    full_path = os.path.join(root, file)
                    # Preserve relative path inside the zip
                    arcname = os.path.relpath(full_path, os.path.dirname(folder_path))
                    zipf.write(full_path, arcname)
        print(f"   ✅ Created ZIP: {os.path.basename(zip_path)}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False
def zip_stats_results_folders(output_base_path: str) -> None:
    """
    Main driver: locate all result folders and zip each one.
    """
    # Ensure the output directory exists
    os.makedirs(output_base_path, exist_ok=True)
    result_folders = find_result_folders(output_base_path)
    if not result_folders:
        print(f"⚠️ No folders starting with 'results_' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = 0
    for folder in result_folders:
        if zip_folder(folder, output_base_path):
            successful += 1
    print(
        f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s)."
    )
if __name__ == "__main__":
    try:
        # Prefer an environment variable; fall back to a global if defined
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        # The script expects a sub‑folder named 'OuroTrace' under OUTPUT_PATH
        target_path = os.path.join(output_root, "")
        zip_stats_results_folders(target_path)
    except Exception as e:
        print(f"❌ An error occurred: {e}")


In [ ]:
# @title Happy Christmas✨
# !rm -r -f FontDiffusion